# MSP算法
by zzh

开源协议：MPL 2.0 （请遵循）

有问题或者发现问题请和作者联系

In [1]:
# graph define
class Edge:
    def __init__(self, u, v, l):
        self.u = u
        self.v = v
        self.l = l
# 定义边集
e = {
    1: Edge('S', 1, 1),
    2: Edge('S', 2, 1),
    3: Edge(1, 3, 2),
    4: Edge(2, 3, 2),
    5: Edge(3, 4, 3),
    6: Edge(3, 5, 3),
    7: Edge(4, 7, 4),
    8: Edge(5, 7, 4),
    9: Edge(7, 'D', 5),
    10: Edge(4, 6, 4),
    11: Edge(5, 8, 4),
    12: Edge(6, 'D', 5),
    13: Edge(8, 'D', 5)
}
# 定义Lambda集合
Lambda = {
    1:  [e[1]],
    2:  [e[2]],
    3:  [e[1], e[2], e[3], e[4]],
    4:  [e[1], e[3], e[5]],
    5:  [e[2], e[4], e[6]],
    6:  [e[1], e[3], e[5], e[6], e[10]],
    7:  [e[12]],
    8:  [e[1] ,e[2] ,e[3] ,e[4] ,e[5] ,e[6] ,e[7] ,e[8] ,e[9] ,e[10] ,e[11] ,e[12] ,e[13]],
    'D':[e[1] ,e[2] ,e[3] ,e[4] ,e[5] ,e[6] ,e[7] ,e[8] ,e[9] ,e[10] ,e[11] ,e[12] ,e[13]]
}

In [2]:
# tools
def vTok(a):
    # 通过value在字典中查找key
    for k,v in e.items():
        if v == a:
            return k

def vTol(v):
    # 通过节点值在在图中寻找

    for i in e:
        if e[i].v == v:
            return e[i].l
    return 0

def union(a, b):
    return [i for i in a if i in b]

In [3]:
# 算子1
def ES(lamd, u, v):
    record = [u]
    res1 = []
    res2 = []
    # 正向搜索
    while len(record) != 0:
        for i in record:
            for ee in lamd:
                if ee.u == i:
                    res1.append(ee)
                    record.append(ee.v)
            record.remove(i)
    
    # 逆向搜索
    record = [v]
    while len(record) != 0:
        for i in record:
            for ee in lamd:
                if ee.v == i:
                    res2.append(ee)
                    record.append(ee.u)
            record.remove(i)
    # 删除多余
    res = [i for i in res1 if i in res2]
    return res


In [4]:
# 算子2
def R(ee):
    record = []
    for key, value in Lambda.items():
        if ee in value:
            record.append(key)
    R_res = []
    for i in e:
        if (e[i].u in record) and (e[i].v in record):
            R_res.append(e[i])
        
    return ES(R_res, ee.v, 'D')

In [5]:
# 算子3
def Comp(es, v):
    ES_temp = es
    l = vTol(v)
    while 1:
        for ee in ES_temp:
            if ee.l < l: # ES(union(R(ee), ES_temp), ee.v, v)
                # 如果为空，移除e
                if len(ES(union(R(ee), ES_temp), ee.v, v)) == 0:
                    ES_temp.remove(ee)
            if ee.l == l:
                if len(ES(R(ee), v, 'D')) == 0:
                    ES_temp.remove(ee)
        ES_res = ES(ES_temp, 'S', v)
        # 重复step2 和 step3,如果ES_temp不在改变则终止，否则ES(ES_temp, 'S', v)赋值给ES_temp
        if ES_res == ES_temp:
            break
        else:
            ES_temp = ES_res
    return ES_temp

In [6]:
# 算子四
def Change(u_v_l):
    R_temp = R(u_v_l)
    u = u_v_l.u
    v = u_v_l.v
    l = u_v_l.l
    E = [e[i] for i in e]
    while 1:
        for a_b_k in R_temp:
            if a_b_k.l > l and a_b_k.l <= vTol('D'):
                ee = []
                for c_d_kk in E:
                    if c_d_kk.l < l:
                        # 参考论文
                        ES_temp = ES(union(R(c_d_kk), Comp(Lambda[a_b_k.v], a_b_k.v)), c_d_kk.v, a_b_k.v)
                        if (u_v_l in ES_temp) and (a_b_k in ES_temp):
                            ee.append(c_d_kk)
                if len(Comp(ES(ee, 'S', u_v_l.u), u_v_l.u)) == 0:
                    R_temp.remove(a_b_k)
        R_res = ES(R_temp, u_v_l.v, 'D')
        if R_res == R_temp:
            break
        else:
            R_temp = R_res

    return R_temp

In [7]:
# (1)
print([vTok(a) for a in ES(Lambda[6], 'S', 6)])
print([vTok(a) for a in ES(Lambda[6], 'S', 7)])
print([vTok(a) for a in ES(Lambda[6], 'S', 7)])
print([vTok(a) for a in ES(Lambda[6], 4, 7)])

[1, 3, 5, 10]
[]
[]
[]


In [8]:
# 算子2测试(2)
for i in e:
    print(i,[vTok(a) for a in R(e[i])])

1 [3, 5, 10, 12]
2 [4, 6, 11, 13]
3 [5, 10, 12]
4 [6, 11, 13]
5 [10, 12]
6 [11, 13]
7 []
8 []
9 []
10 [12]
11 [13]
12 []
13 []


In [9]:
# 算子3测试
[vTok(a) for a in Comp(Lambda[5], 5)]

[2, 4, 6]

In [10]:
# 算子4测试
Change(e[6])

[]